In [66]:
#from methods import *
import re
import benepar
import pandas as pd
import nltk
import nltk.draw
from nltk.tree import Tree
from nltk.draw.tree import TreeView
from nltk.stem import WordNetLemmatizer as wln
import os
from IPython.display import display
# import spacy
from  icecream import ic
from scipy.stats import chisquare
from nltk.corpus import wordnet as wn
from flair.data import Sentence
from flair.nn import Classifier
import numpy as np
from nltk.data import find
from nltk.test.gensim_fixt import setup_module
import gensim
nltk.download('wordnet')

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
sentence_enders = {".", '?', '!', ':', ';'}
sentential_tags = {'S', 'SBAR', 'SINV', 'SBARQ', 'SQ'}
negators = {'not', 'n\'t'}
verb_tags = {'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'VB', 'MD'}
adverb_tags = {'RB', 'RBR', 'RBS', 'WRB'}
adj_tags = {'JJ', 'JJR', 'JJS'}
noun_tags = {'NN', 'NNP', 'NNS', 'NNPS'}
tensed = {'have', 'do', 'will', 'would', 'can', 'could', 'may', 'might', 'should', 'ought', 'must', 'shall', 'be', 'dare'}
wh_tags = {'WDT', 'WP', 'WP$', 'WRB'}
mod_tags = {'JJ', 'JJR', 'JJS', 'DT', 'PDT', 'CD'}
prep_tags = {'IN', 'TO'}

setup_module()
tagger = Classifier.load('sentiment')
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [ ]:
# takes in an array of english words and returns true if one of a 
# set of words is present
def english_indicator(text, options=['only', 'just']):
  for w in text:
    if w in options:
      return False
  return True

# takes in a line of english text, splits the line into paragraphs
# returns a line splinto sentences as an array of array of strings
# if containing all alphanumeric characters, or if a specified
# other condition is met
def english_processor(text, specific_processor=None):
  res = []

  #https://stackoverflow.com/questions/25735644/python-regex-for-splitting-text-into-sentences-sentence-tokenizing
  sentences = re.split(r'(?<!\w\.\w.)(?<!\b[A-Z][a-z]\.)(?<![A-Z]\.)(?<=\.|\?)\s|\\n', text)

  sentences = [s.split(' ') for s in sentences]


  sentences = [[w.lower() for w in s if w not in { '\n', '<p>', '', ',', '.', '\"', '\'', '?', '!', ':', ';'}] for s in sentences]

  sentences = [s for s in sentences if '@' not in s]

  sentences = [s for s in sentences]

  for s in sentences:
    if not s:
      continue
    if '@' in s:
      continue
    joined = ''.join(s)
    if '#' in joined:
      continue
    if ':' in joined:
      continue
    if ';' in joined:
      continue
    if '-' in joined:
      continue
    res.append(s)
  return res

def new_processor(text, specific_processor=None):
  res = nltk.sent_tokenize(text)
  ic(res)
  return res


# takens in the path to a .txt file, a function that takes in a 
# line containing multiple sentences and returns array of array
# of words in sentences, and a function that return true iff a 
# sentence is valid
def valid_sentences(textpath, processor, indicator):
  res = []
  i = 0
  with open(textpath, 'r') as f:
    for line in f:
      #print(line)
      sentences = [s for s in processor(line) if indicator(s)]
      res = res + sentences
  return res

def pp(t):
  return str(t.flatten()).replace('\n', ' ').replace('   ', ' ')

def text(t):
  return ' '.join(t.leaves())

def conjoin(a, b, left):
  if left:
    return Tree.fromstring(b.label() + ' (' + text(a) + ' ' + text(b) + ')')
  else:
    return Tree.fromstring(a.label() + ' (' + text(a) + ' ' + text(b) + ')')
  
def lowest_pos(t):
  return [s for s in t.subtrees(lambda t: t.height() == 2)]

def lowest_leaves(t):
  return [t[s] for s in t.subtrees(lambda t: t.height() == 2)]
#returns if corrective, anchored (verb negated)
def find_s(t, pos0):
  sentential_tags = {'S', 'SBAR', 'SINV', 'SBARQ', 'SQ'}
  negators = {'not', 'n\'t'}
  verb_tags = {'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
  tensed = {'have', 'do', 'will', 'would', 'can', 'could', 'may', 'might', 'should', 'ought', 'must', 'shall', 'be'}

  try:
    while t[pos].label() not in sentential_tags:
      pos = pos[:-1]
  except:
    return (False, False)
  poses = lowest_pos(t)
  i = None
  for k in range(poses.index(pos0)):
    if text(t[poses[k]]) in negators:
      i = k
      break
  if i == None:
    return (False, False)
  try:
    temp = wln().lemmatize(text(t[poses[i-1]]),pos='v')
    if t[poses[i-1]].label() in verb_tags and temp in tensed and temp != 'VB':
      return (True, True)
    return (True, False)
  except:
    return (False, False)

def funnel(tag):
  if tag in verb_tags:
    return "V"
  elif tag in adverb_tags:
    return 'R'
  elif tag in adj_tags:
    return 'J'
  elif tag in noun_tags:
    return 'N'
  elif tag in sentential_tags:
    return 'S'
  else:
    return 'X'


def sent_apply(r):
  s0 = Sentence(r)
  tagger.predict(s0)
  sentiment = s0.labels[0].value
  score = s0.labels[0].score
  if sentiment == 'NEGATIVE':
    return -1 * score
  return score

def cosine_apply(p, q):
  if ' ' in p or ' ' in q:
    return np.NaN
  return model.wv.similarity(p,q)
  


In [54]:
print("VBD" in verb_tags)

True


In [55]:
parser = benepar.Parser("benepar_en3")
ans = valid_sentences('COCA/text_academic_rpe/w_acad_1990.txt', english_processor, english_indicator)[:100]
df = pd.DataFrame({"sentence":ans})
tree_gen = parser.parse_sents([benepar.InputSentence(s) for s in ans])
trees = []
for t in tree_gen:
  trees.append(t)

/Users/oliverweizel/.pyenv/versions/3.12.7/lib/python3.12/site-packages/benepar/parse_chart.py:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
  

In [56]:
txt1 = "He does not walk to the pool but runs"
bs = parser.parse_sents(txt1)
ok = []
for o in bs:
  o.pretty_print()

         TOP                                   
          |                                     
          S                                    
  ________|________                             
 |                 VP                          
 |             ____|________________________    
 |            VP                        |   |  
 |    ________|________                 |   |   
 |   |    |            VP               |   |  
 |   |    |    ________|___             |   |   
 |   |    |   |            PP           |   |  
 |   |    |   |     _______|___         |   |   
 NP  |    |   |    |           NP       |   VP 
 |   |    |   |    |        ___|___     |   |   
PRP VBZ   RB  VB   IN      DT      NN   CC VBZ 
 |   |    |   |    |       |       |    |   |   
 He does not walk  to     the     pool but runs



In [57]:
sentential_tags = {'S', 'SBAR', 'SINV', 'SBARQ', 'SQ'}
negators = {'not', 'n\'t'}

In [58]:
parent_tree = []
parent_text = []
parent_type = []
left_tree = []
left_text = []
left_type = []
cc_tree = []
cc_text = []
right_tree = []
right_text = []
right_type = []
corrective = []
anchored = []
clausal_coord = []
base_text = []


for t in trees:
  #ic('full: '+ pp(t))
  if 'the pain and anger' in text(t):
    ic(t)
  for tp in t.treepositions():
    if isinstance(t[tp], str):
      continue
    if t[tp].label() == 'CC':
      parent = t[tp[:-1]]
      relative_tp = tp[-1]
      # ic(relative_tp)
      # ic(parent.treepositions())
      if relative_tp + 1 >= len(parent) or relative_tp - 1 <= -1:
        #print("bad conjunction")
        #ic(pp(parent))
        continue
      # try:
      #   _ = parent[relative_tp + 1]
      #   _ = parent[relative_tp - 1]
      # except:
      #   print("bad conjunction")
      #   ic(pp(parent))
      #   continue
      #ic('parent: ' + pp(parent))
      left = parent[relative_tp - 1]
      #ic('left: ' + pp(left))
      conj = parent[relative_tp]
      #ic('conj: ' + pp(conj))
      #ic('conjunction: ' + ' '.join(conj.leaves()))
      right = parent[relative_tp + 1]
      #ic('right: ' + pp(right))
      cc = False
      if left.label() in sentential_tags and right.label() in sentential_tags:
        cc = True
      
      if 'only' in left.leaves() or 'just' in left.leaves():
        continue
      if text(left) == 'but':
        continue
      if left.label() == 'CC':
        continue
      # try:
      #   farleft = parent[relative_tp - 2]
      #   if text(farleft) == 'not' and text(conj) == 'but':
      #     corrective.append(True)
      #     left = conjoin(farleft, left, True)
      #   else:
      #     corrective.append(False)
      # except:
      #   corrective.append(False)
      corr, anch = find_s(t, tp)
      # do the rest of this oliver!!!
      try:
        farright = parent[relative_tp + 2]
        if right.label() == 'ADVP' and farright.label() != 'ADVP':
          right = conjoin(right, farright, False)
      except:
        pass
      

      parent_tree.append(parent)
      parent_text.append(text(parent))
      parent_type.append(parent.label())
      left_tree.append(left)
      left_text.append(text(left))
      left_type.append(left.label())
      right_tree.append(right)
      right_text.append(text(right))
      right_type.append(right.label())
      cc_tree.append(conj)
      cc_text.append(text(conj))
      corrective.append(corr)
      anchored.append(anch)
      clausal_coord.append(cc)
      base_text.append(text(t))


ic| t: Tree('TOP', [Tree('SBAR', [Tree('WHNP', [Tree('-LRB-', ['-LRB-']), Tree('S', [Tree('SBAR', [Tree('WHNP', [Tree('WP', ['what'])]), Tree('S', [Tree('VP', [Tree('VBD', ['brought']), Tree('NP', [Tree('PRP', ['him'])]), Tree('NP', [Tree('NN', ['shame'])])])])]), Tree('VP', [Tree('VBD', ['was']), Tree('RB', ['not']), Tree('NP', [Tree('NP', [Tree('DT', ['the']), Tree('NN', ['pain']), Tree('CC', ['and']), Tree('NN', ['anger'])]), Tree('CC', ['but']), Tree('NP', [Tree('NP', [Tree('PRP$', ['his']), Tree('NN', ['trading'])]), Tree('PP', [Tree('IN', ['on']), Tree('NP', [Tree('PRP', ['them'])])]), Tree('PP', [Tree('IN', ['for']), Tree('SBAR', [Tree('WHNP', [Tree('WP', ['what'])]), Tree('S', [Tree('VP', [Tree('VBD', ['turned']), Tree('PRT', [Tree('RP', ['out'])]), Tree('S', [Tree('VP', [Tree('TO', ['to']), Tree('VP', [Tree('VB', ['be']), Tree('NP', [Tree('DT', ['a']), Tree('JJ', ['mere']), Tree('CD', ['twenty']), Tree('NNS', ['bucks'])])])])])])])])])])])])])])])])


In [63]:
df = pd.DataFrame({'parent_tree':parent_tree, 'parent_text':parent_text, 'parent_type':parent_type, 'left_tree':left_tree, 'left_text':left_text, 'left_type':left_type, 'right_tree':right_tree, 'right_text':right_text, 'right_type':right_type, 'cc_tree':cc_tree, 'cc_text':cc_text})
df['base_text'] = base_text
df['parent_funnel'] = df.apply(lambda row: funnel(row['parent_type']), axis=1)
df['left_funnel'] = df.apply(lambda row: funnel(row['left_type']), axis=1)
df['right_funnel'] = df.apply(lambda row: funnel(row['right_type']), axis=1)
df['same_type'] = df.apply(lambda row: row['left_type'] == row['right_type'], axis=1)
df['same_funnel'] = df.apply(lambda row: row['left_funnel'] == row['right_funnel'], axis=1)
df['clausal_coord'] = clausal_coord
df['corrective'] = corrective
df['left_sent'] = df.apply(lambda row: sent_apply(row['left_text']), axis=1)
df['base_sent'] = df.apply(lambda row: sent_apply(row['base_text']), axis=1)
df['right_sent'] = df.apply(lambda row: sent_apply(row['right_text']), axis=1)
print(df['cc_text'].value_counts())
res = chisquare(df['cc_text'].value_counts())
print(res)
print(df.head())

cc_text
and    34
but     8
or      2
nor     1
Name: count, dtype: int64
Power_divergenceResult(statistic=np.float64(63.888888888888886), pvalue=np.float64(8.669650361275029e-14))
                                         parent_tree  \
0  [[[(PRP i)], [(MD can), (VP\n  (VB see)\n  (AD...   
1  [[[came], [(RB right), (RB out)]], [and], [[as...   
2  [[[(PRP$ our), (NN color)], [(VBD had), (VP (V...   
3  [[[smiled], [(IN at), (NP (DT the) (NN man))],...   
4  [[[(RB certainly)], [(PRP i)], [(RB also)], [(...   

                                         parent_text parent_type  \
0  i can see now that possibly she was but then a...           S   
1  came right out and asked me to feel sorry for ...          VP   
2  our color had brought this man anguish and if ...           S   
3  smiled at the man to let him know he should n'...          VP   
4  certainly i also judge him a coward but in tha...           S   

                                           left_tree  \
0  [[[i]], [[can]

In [64]:
df.to_csv('out3.csv', index=False)